In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

### 1. Aggregate data to specifc time interval

In [33]:
def aggregate_data(data, aggregate_len):
    data = data.groupby(np.arange(len(data))//aggregate_len).agg( {'timeStamp': 'first', 'flow':lambda x: x.sum(min_count=1), 
                                                                       'siteId':'first', 'description':'first', 
                                                                       'lat':'first', 'lon':'first', 
                                                                       'easting': 'first', 'northing':'first'})
    return data

### 2. Add stage to each data

In [42]:
def add_stage(data):    
    data['date'] = data['timeStamp'].str.split(' ').str[0]
    data['date'] =  pd.to_datetime(data['date'])
    data['stage'] = 0
    for i in range(len(data)):
        if data['date'][i] < datetime(2023, 6, 1):
            data.loc[data.index == i,'stage'] = 'Before LEZ'
        else:
            data.loc[data.index == i,'stage'] = 'After LEZ'
    return data

In [44]:
sample_list = pd.read_csv('detector_671.csv').siteId.tolist()
data_list = []
for i in sample_list:
    data = pd.read_csv('SCOOT_671/' + i + '&2023_03_01-08_31&30.csv')
    data = aggregate_data(data, 2)
    data = add_stage(data)
    data_list.append(data)
data_mix = pd.concat(data_list).reset_index(drop=True)

In [51]:
fig = px.box(data_mix, x='siteId' ,y="flow", color="stage", width=1000, height=500)
fig.show()

In [ ]:
# Plot 671 data in each box with timestamp as x-axis
# Plot all before/after LEZ in each box with 671 data as x-axis

# seperate data into within or outside LEZ as x-axis, and plot two boxs